访问本地ollama模型

In [18]:
from langchain.llms import Ollama

# 选择 LLM，如 "mistral" 或 "llama2"
llm = Ollama(model="deepseek-r1:32b", base_url="http://localhost:6399")

# 生成文本
response = llm("你是谁")
print(response)


<think>

</think>

您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。


访问本地ollama模型+资料库

In [69]:
#公共结构定义

from langchain.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    JSONLoader,
    UnstructuredMarkdownLoader
)
from langchain.embeddings import OllamaEmbeddings
import numpy as np

# 根据文档类型选择参数
splitter_configs = {
        "general": {
            "chunk_size": 1000,
            "chunk_overlap": 200
        },
        "technical": {
            "chunk_size": 1500,
            "chunk_overlap": 300,
            "separator": "\n\n"
        },
        "code": {
            "chunk_size": 800,
            "chunk_overlap": 100,
            "separator": "\n\n"
        },
        "markdown": {
            "chunk_size": 1000,
            "chunk_overlap": 150,
            "separator": "\n## "    # Markdown 二级标题作为分隔符
        },
        "news": {
            "chunk_size": 500,      # 减小块大小
            "chunk_overlap": 50,     # 适当的重叠
            "separator": "\n"        # 使用换行符作为分隔符
        }
    }

# 文件类型到加载器的映射
LOADER_MAPPING = {
    '.txt': TextLoader,
    '.md': UnstructuredMarkdownLoader,
    '.pdf': PyPDFLoader,
    '.csv': CSVLoader,
    '.json': JSONLoader
}

class NormalizedOllamaEmbeddings(OllamaEmbeddings):
    def embed_documents(self, texts):
        embeddings = super().embed_documents(texts)
        # L2 归一化
        normalized = [embedding/np.linalg.norm(embedding) for embedding in embeddings]
        return normalized
    
    def embed_query(self, text):
        embedding = super().embed_query(text)
        # L2 归一化
        return embedding/np.linalg.norm(embedding)



In [70]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    JSONLoader,
    UnstructuredMarkdownLoader
)
from langchain.text_splitter import CharacterTextSplitter
from typing import List, Dict

def load_documents_from_directory(
    directory_path: str,
    db_path: str,
    doc_type: str = "news",
    supported_extensions: List[str] = ['.txt', '.md', '.py']
) -> None:
    # 设置嵌入模型
    embedding = NormalizedOllamaEmbeddings(
        model="bge-m3:latest",
        base_url="http://localhost:6399"
    )
    
    # 获取分割器配置
    splitter_config = splitter_configs.get(doc_type, splitter_configs["general"])
    text_splitter = CharacterTextSplitter(**splitter_config)
    
    # 存储所有文档
    all_documents = []
    file_count = 0
    
    try:
        # 遍历目录
        for root, _, files in os.walk(directory_path):
            for file in files:
                if any(file.endswith(ext) for ext in supported_extensions):
                    file_path = os.path.join(root, file)
                    try:
                        # 添加调试信息
                        print(f"\n开始处理文件: {file_path}")
                        print(f"文件大小: {os.path.getsize(file_path)} bytes")
                        
                        # 尝试直接读取文件内容
                        with open(file_path, 'r', encoding='utf-8') as f:
                            raw_content = f.read()
                            print(f"原始文件内容长度: {len(raw_content)} 字符")
                        
                        # 加载文档
                        file_extension = os.path.splitext(file_path)[1].lower()
                        if file_extension in LOADER_MAPPING:
                            loader_class = LOADER_MAPPING[file_extension]
                            loader = loader_class(file_path, encoding='utf-8')  # 显式指定编码
                            documents = loader.load()
                            print(f"加载后的文档数量: {len(documents)}")
                            
                            if len(documents) == 0:
                                print(f"警告: 文档加载后为空")
                                continue
                                
                            # 分割文档
                            texts = text_splitter.split_documents(documents)
                            print(f"分割后的文本块数量: {len(texts)}")
                            
                            if len(texts) == 0:
                                print(f"警告: 文本分割后为空")
                                continue
                                
                            all_documents.extend(texts)
                            file_count += 1
                            print(f"成功加载文件: {file_path}")
                            
                    except Exception as e:
                        print(f"处理文件 {file_path} 时出错: {str(e)}")
                        print(f"错误类型: {type(e).__name__}")
                        import traceback
                        print(f"详细错误信息: {traceback.format_exc()}")
                        
        print(f"\n总共处理了 {file_count} 个文件")
        print(f"总共获得 {len(all_documents)} 个文本块")
        
        # # 创建或更新向量数据库
        # try:
        #     # 尝试加载现有数据库
        #     vector_db = FAISS.load_local(
        #         db_path,
        #         embeddings=embedding,
        #         allow_dangerous_deserialization=True
        #     )
        #     print("加载现有数据库成功，添加新文档...")
        #     vector_db.add_documents(all_documents)
        # except:
        print("创建新的向量数据库...")
        print(all_documents)
        vector_db = FAISS.from_documents(all_documents, embedding)
        
        # 保存数据库
        vector_db.save_local(db_path)
        print("成功保存向量数据库")
        
        # 测试搜索
        results = vector_db.similarity_search("黄金", k=7)
        print("\n测试搜索结果:")
        for i, doc in enumerate(results, 1):
            print(f"\n文档 {i}:")
            print(f"来源: {doc.metadata['source']}")
            print(f"内容: {doc.page_content[:100]}...")
        
    except Exception as e:
        print(f"处理过程中出错: {e}")


# 配置参数
directory_path = "test_file"  # 文档目录
doc_type = "news"            # 文档类型
supported_extensions = [      # 支持的文件类型
    '.txt',
    '.md',
    '.pdf',
    '.json',
    '.csv'
]
    
# 执行导入
load_documents_from_directory(
    directory_path=directory_path,
    db_path="faiss_index",
    doc_type=doc_type,
    supported_extensions=supported_extensions
)


开始处理文件: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt
文件大小: 13472 bytes
原始文件内容长度: 9906 字符
加载后的文档数量: 1
分割后的文本块数量: 23
成功加载文件: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt

开始处理文件: test_file/金价向上、金店向下：金饰品牌加速关店.txt
文件大小: 4963 bytes
原始文件内容长度: 1857 字符
加载后的文档数量: 1
分割后的文本块数量: 5
成功加载文件: test_file/金价向上、金店向下：金饰品牌加速关店.txt

总共处理了 2 个文件
总共获得 28 个文本块
创建新的向量数据库...
[Document(metadata={'source': 'test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt'}, page_content='# Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）描述\n## 表名\nRank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）\n## 字段说明\n### 维度字段（共49个）\n1. **day**：天，分区字段。\n2. **hour**：小时，分区字段。\n3. **minute**：分钟，分区字段。\n4. **media_id**：媒体id，分区字段。\n5. **adslot_id**：栏位id，全局统一的广告栏位id。\n6. **user_id**：广告主用户id，全局统一的广告栏位id。\n7. **unit_id**：单元id，全局统一的广告栏位id。\n8. **ideaid**：创意id，全局统一的广告栏位id。\n9. **adslot_type**：栏位类型，1代表列表页，2代表详情页，3代表互动，4代表开屏，5代表横幅，6代表视频，7代表激励(应用商城)。'), Document(metadata={'source': 'test_file/Rank算法粗排实时主题表（

In [55]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# 设置嵌入模型
embedding = OllamaEmbeddings(
    model="bge-m3:latest",
    base_url="http://localhost:6399"
)

# 加载文档
loader = TextLoader('test_file/金价向上、金店向下：金饰品牌加速关店.txt')  # 替换为你的文档路径
documents = loader.load()

# 分割文档
# 根据文档类型选择参数
splitter_config = splitter_configs.get("news", splitter_configs["general"])
print(splitter_config)

# 创建分隔器
text_splitter = CharacterTextSplitter(**splitter_config)

texts = text_splitter.split_documents(documents)
print(texts)

# 创建向量数据库
vector_db = FAISS.from_documents(texts, embedding)

# 保存到本地（注意路径）
vector_db.save_local("faiss_index")

{'chunk_size': 500, 'chunk_overlap': 50, 'separator': '\n'}
[Document(metadata={'source': 'test_file/金价向上、金店向下：金饰品牌加速关店.txt'}, page_content='与上一轮金价上涨期间，黄金投资、金饰消费均出现增长不同的是，如今金价越涨越高，黄金首饰却卖不动了。\n世界黄金协会近日公布的一组数据显示，过去一年全球金饰消费同比下降11%，其中，中国市场2024年金饰消费量同比下降24%。\n行业内多数金店对此感同身受，周大福、周生生、老凤祥过去一年出现销量、业绩的双下滑。发生了什么？未来金饰需求会有改善吗？\n金价上涨，金饰难卖\n在全球央行“购金热”的推动下，金价持续上涨，2月25日现货黄金一度站上2940美元/盎司，今年以来累计涨幅超过11%，时间略微拉长，自2024年以来，现货黄金累计涨幅超42%。\n黄金首饰兼具消费与投资属性，在上一轮“黄金大牛市”中，消费者对黄金首饰的购买量持续上涨。中国黄金协会数据显示，2010年我国黄金首饰需求357吨，随后一路上涨，到2013年首饰用金超716吨，需求量几乎翻了一倍。\n2024年国际金价全年涨幅超25%，但我国黄金市场的首饰用金需求却逆向而行，比上一年减少了151吨。'), Document(metadata={'source': 'test_file/金价向上、金店向下：金饰品牌加速关店.txt'}, page_content='2024年国际金价全年涨幅超25%，但我国黄金市场的首饰用金需求却逆向而行，比上一年减少了151吨。\n具体到金饰企业，周大福公布的经营数据显示，2024年4-9月，周大福珠宝中国内地同店销售额下滑25.4%，按件数算的销量减少了30%；中国香港及中国澳门的同店销售额下滑30.8%，销售件数同比减少33%。虽然毛利率有所提升，达到了31.4%，但周大福整体销售额同比下降20.4%至394亿港元。\n2024年10-12月，周大福同店销售跌幅收窄，但销售额同比降幅仍超过16%，同店销量同比下滑幅度仍接近30%。\n销量下滑带来的一个后果是存货周转天数变长，周大福财报数据显示，2024年9月底的存货周转天数超过400天，2023年初的存货周转天数则为300天左右。\n有金店店员

In [72]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings

# 使用 Ollama 的嵌入模型
embedding = NormalizedOllamaEmbeddings(
    model="bge-m3:latest",  # 或其他你想使用的模型
    base_url="http://localhost:6399"  # 使用你的 Ollama 服务端口
)

# 创建或加载向量数据库
vector_db = FAISS.load_local(
    "faiss_index", 
    embeddings=embedding,
    allow_dangerous_deserialization=True  # 添加这个参数
)

# 语义搜索
query = "为什么金价向上、金店向下？"
# 增加 k 和 score_threshold
docs_and_scores = vector_db.similarity_search_with_score(
    query,
    k=7,                    # 返回更多结果
    score_threshold=0.9     # 设置相似度阈值
)

print("\n=== 搜索结果 ===")
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n文档 {i} (相似度分数: {score}):")
    print(f"来源: {doc.metadata['source']}")
    print(f"内容: {doc.page_content[:200]}...")
    print("-" * 50)



=== 搜索结果 ===

文档 1 (相似度分数: 0.6504273414611816):
来源: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt
内容: # Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）描述
## 表名
Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）
## 字段说明
### 维度字段（共49个）
1. **day**：天，分区字段。
2. **hour**：小时，分区字段。
3. **minute**：分钟，分区字段。
4. **m...
--------------------------------------------------

文档 2 (相似度分数: 0.8344680666923523):
来源: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt
内容: 30. **cvr_event_adv**：优化退费率开关，0代表关闭，1代表打开，默认0关闭。
31. **media_set_id**：媒体集id。
32. **bidding_type**：竞价类型，-1代表默认值，1代表客户端竞价，2代表固定CPM。
33. **return_hist**：竞价类型，-1代表默认值，1代表客户端竞价，2代表固定CPM。
34. **put_type**：拉...
--------------------------------------------------

文档 3 (相似度分数: 0.839082658290863):
来源: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt
内容: 38. **arpu**：标记rta参竞并且已安装的拉新，不去重累加，arpu/10后单位是分。
39. **incr_show**：作为维度数据，incr_show=0是真实曝光数据，incr_show=1是incr_show数据。
40. **arpu_model_name**：arpu模型名称。
41. **c

In [79]:
#直接使用 LLM + 模板

from langchain.callbacks.base import BaseCallbackHandler
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

# 处理搜索结果
def get_answer(query, search_results):
    # 合并上下文
    context = "\n\n".join([doc.page_content for doc in search_results])
    
    # 生成完整提示
    final_prompt = prompt.format(
        context=context,
        question=query
    )
    
    # 获取回答
    for chunk in llm.stream(final_prompt):
        print(chunk, end="", flush=True)


class StreamingHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        print(token, end="", flush=True)

# 创建 LLM
llm = Ollama(
    model="deepseek-r1:32b",
    base_url="http://localhost:6399",
    #callbacks=[StreamingHandler()]
)

# 创建提示模板
template = """基于以下信息回答问题：

背景信息：
{context}

问题：{question}

请给出详细的回答："""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

query = "rank表的栏位信息"

get_answer(query, vector_db.similarity_search(query, k=7))

<think>
好的，我现在需要帮助用户回答关于“rank表的栏位信息”的问题。首先，我得仔细阅读提供的背景信息，理解表格中的字段及其描述。

用户的问题是关于Rank算法粗排实时主题表（dl_...）里的栏位信息。所以我应该关注与栏位相关的维度和指标。

从背景资料中，我看到维度部分提到了“栏位信息”，具体包括栏位ID、类型、层级、名称和来源。这些应该是用来描述广告展示的位置或环境的。我需要将每个栏位属性详细解释清楚，比如它们各自的作用是什么，如何帮助分析数据。

接下来是指标部分，主要涉及点击率预测（如as_ctr_sum）、转化率预测（as_cvr_sum）以及收入相关的计算（总收入、CPC收入、DSP收入）。这些指标能反映栏位的性能和贡献，所以也需要详细说明每个指标的含义及其重要性。

在组织回答时，我应该分维度和指标两部分，每部分下再细分具体的字段。这样结构清晰，方便用户理解。同时，要确保语言简洁明了，避免使用过于专业的术语，让用户容易消化信息。

最后，检查是否有遗漏的信息，比如每个栏位来源的具体含义（如1代表联盟、28代表内部等），这些细节对用户来说可能很重要，需要明确说明。
</think>

Rank表的栏位信息主要包括以下内容：

### 1. 栏位维度
- **栏位ID (position_id)**：标识广告展示的位置或环境。
- **栏位类型 (position_type)**：描述广告位置的具体类型（如搜索结果、推荐列表等）。
- **栏位层级 (level)**：指示栏位的层次结构，通常用于多级流量分配。
- **栏位名称 (position_name)**：栏位的人类可读名称。
- **栏位来源 (adsrc)**：标识广告的来源或渠道（如联盟、内部等）。

### 2. 栏位相关指标
- **点击率预测 (as_ctr_sum)**：
  - 老口径计算方式：`sum(show * exp_ctr)`
  - 新口径计算方式：`sum((impression - impression * incr_show) * exp_ctr)`
  - 描述广告点击的可能性。
  
- **转化率预测 (as_cvr_sum)**：
  - 计算方式：`sum(click * exp_cvr)`
  - 表示用户在点击后完成目标操作的概率。

